Fixed Errors - 
1. Exchanged latitudes and longotudes
2. Replaced Nan latitudes and longitudes with zeroes

In [12]:
import os
import glob
import plotly
import osmnx as ox
import plotly.express as px
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from ipypb import track
from shapely.geometry import Polygon, MultiPolygon
from geopy.distance import vincenty

In [11]:
Data_Path = "C:\\Users\\sahit\\ADES\\ADES\\DataSource"

Potential Errors -
1. Duplicate latitude and longitude 
2. Zero latitude and longitude 

Possible approaches - 
1. Check the difference in latitude and longitude and it must be meaningful 
2. Check the latitude difference with the chainage of the bridge part that is calculated. 
3. No IDs for some of the bridges (Column: name)
4. Is it true that one road has one name?


In [120]:
bridges = pd.read_excel(Data_Path + '\\BMMS_overview_SortedRoad.xlsx')

In [121]:
bridges.head(5)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.00,C,101102,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,14.65,1986.0,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702083,90.515917,bcs1
4,N1,10.543,Box Culvert,LRP010a,KATCHPUR BOX CULVERT,8.00,B,101106,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.543,21.70,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.702056,90.528194,bcs1


In [19]:
bridges["type"].unique()

array(['Box Culvert', 'PC Girder Bridge', 'RCC Girder Bridge', 'PC Box',
       'Slab Culvert', 'RCC Bridge', 'Arch Masonry',
       'Truss With Timber Deck', 'Steel Beam & RCC Slab',
       'Baily with Steel Deck', 'Truss with RCC Slab',
       'Truss with Steel Deck', 'Baily with Timber Deck', 'Pipe Culvert'],
      dtype=object)

In [122]:
for i in range(len(bridges)):
    if bridges['lat'][i]>90 :
        c= bridges['lat'][i] 
        bridges['lat'][i] = bridges['lon'][i] 
        bridges['lon'][i] =c
        

C:\Users\sahit\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\sahit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [123]:
bridges['lat'] = bridges['lat'].fillna(0)
bridges['lon'] = bridges['lon'].fillna(0)

In [126]:
bridges['lon'].sort_values()

11526     0.000000
8032      0.000000
12821     0.000000
7697      0.000000
7689      0.000000
12824     0.000000
7663      0.000000
11680     0.000000
11679     0.000000
20296     0.000000
20294     0.000000
11676     0.000000
20291     0.000000
20288     0.000000
21404     0.000000
21045     0.000000
21040     0.000000
16508     0.000000
20485     0.000000
11529     0.000000
11532     0.000000
1996      0.000000
10102     0.000000
11964     0.000000
8034      0.000000
15921     0.000000
15930     0.000000
15936     0.000000
12678     0.000000
10722     0.000000
           ...    
5612     92.411000
5610     92.411500
5611     92.414750
5613     92.417444
5614     92.418705
5615     92.424861
5616     92.427111
11958    92.428528
5633     92.431056
5632     92.439139
5631     92.441722
5630     92.446972
5617     92.448333
5618     92.453056
5629     92.453361
5619     92.456361
5628     92.463611
5627     92.469389
5620     92.470750
5626     92.474444
5624     92.476038
5625     92.

In [81]:
def distance_calc (lat1, lon1, lat2, lon2):
    start = (lat1, lon1)
    stop = (lat2, lon2)

    return vincenty(start, stop).kilometers

In [127]:
lon = pd.concat([bridges['lon'].shift(), bridges.loc[1:, 'lon']], axis=1, ignore_index=True)
lat = pd.concat([bridges['lat'].shift(), bridges.loc[1:, 'lat']], axis=1, ignore_index=True)

In [129]:
lat.columns = ['startlat', 'stoplat']
lon.columns = ['startlon', 'stoplon']

In [132]:
lon.drop(lon.index[0],inplace= True)
lat.drop(lat.index[0],inplace= True)

In [140]:
lon = lon.reset_index(drop = True)
lat = lat.reset_index(drop = True)

In [135]:
lat.head()

,index,startlat,stoplat
0,1,23.702889,23.693611
1,2,23.693611,23.704583
2,3,23.704583,23.702083
3,4,23.702083,23.702056
4,5,23.702056,23.699833


In [136]:
lon.head()

,index,startlon,stoplon
0,1,90.450389,90.478833
1,2,90.478833,90.518833
2,3,90.518833,90.515917
3,4,90.515917,90.528194
4,5,90.528194,90.530722


In [143]:
bridges['dist']= np.zeros

In [ ]:
for i in range(len(lon)):
        bridges['dist'][i] = distance_calc(lat['startlat'][i], lon['startlon'][i],lat['stoplat'][i], lon['stoplon'][i] )

C:\Users\sahit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning:

Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.

C:\Users\sahit\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

